In [78]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import TextLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

In [80]:
# Load the embedding and LLM model
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name = "gpt-4o", max_tokens = 1000)

In [81]:
# Load data
loader = TextLoader("/Users/rianrachmanto/miniforge3/project/RAG_Drill_Report/data/well_remark.txt")
docs = loader.load()

In [82]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [83]:
db = Chroma.from_documents(documents, embedding = embeddings_model, persist_directory="/Users/rianrachmanto/miniforge3/project/RAG_Drill_Report/test_index")
db.persist()

In [84]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [85]:
chain = load_qa_chain(llm, chain_type="stuff")

In [86]:
# # A utility function for answer generation
# def ask(question):
#     context = retriever.get_relevant_documents(question)
#     answer = (chain({"input_documents": context,"question": question}, return_only_outputs=True))['output_text']
#     return answer


In [87]:
# A utility function for answer generation
def ask(question):
    # Define the prompt template
    prompt_template = """Answer the following question based only on the provided context.
    Please provide with detail answers. 
    If you don't know the answer, just say you don't know. Don't try to make up an answer.
    
    Context:
    {context}
    
    Question: {question}
    
    Answer: Let's approach this step by step:"""
    
    # Retrieve relevant documents
    context = retriever.get_relevant_documents(question)
    
    # Format the prompt with the context and question
    prompt = prompt_template.format(context=context, question=question)
    
    # Generate the answer using the prompt
    answer = (chain({"input_documents": context, "question": prompt}, return_only_outputs=True))['output_text']
    
    return answer

In [88]:
user_question = "List all the date of setting plug"
answer = ask(user_question)
print("Answer:", answer)

Answer: Based on the provided context, the plug was set on the date 01-Feb-2014.
